In [1]:
import pandas as pd
import scipy.sparse as sparse
import numpy as np
from scipy.sparse.linalg import spsolve

In [131]:
grouped_purchased = pd.read_csv('cvs_data_file/online-retail.csv', header = 0)
grouped_purchased.head(n=10)

,CustomerID,StockCode,Quantity
0,12346,23166,1
1,12347,16008,24
2,12347,17021,36
3,12347,20665,6
4,12347,20719,40
5,12347,20780,12
6,12347,20782,6
7,12347,20966,10
8,12347,21035,6
9,12347,21041,12


In [3]:
customers = list(np.sort(grouped_purchased.CustomerID.unique())) # Get our unique customers
products = list(grouped_purchased.StockCode.unique()) # Get our unique products that were purchased
quantity = list(grouped_purchased.Quantity) # All of our purchases

rows = grouped_purchased.CustomerID.astype('category', categories = customers).cat.codes 
# Get the associated row indices
cols = grouped_purchased.StockCode.astype('category', categories = products).cat.codes 
# Get the associated column indices
purchases_sparse = sparse.csr_matrix((quantity, (rows, cols)), shape=(len(customers), len(products)))

C:\Users\Office\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: specifying 'categories' or 'ordered' in .astype() is deprecated; pass a CategoricalDtype instead
  """
C:\Users\Office\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: specifying 'categories' or 'ordered' in .astype() is deprecated; pass a CategoricalDtype instead
  import sys


In [4]:
print(purchases_sparse.A)

[[ 1  0  0 ...  0  0  0]
 [ 0 24 36 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]
 ...
 [ 0  0  0 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]]


In [5]:
matrix_size = purchases_sparse.shape[0]*purchases_sparse.shape[1] # Number of possible interactions in the matrix
num_purchases = len(purchases_sparse.nonzero()[0]) # Number of items interacted with
sparsity = 100*(1 - (num_purchases/matrix_size))
sparsity

98.32190920694744

In [6]:
import random

In [7]:
def make_train(ratings, pct_test = 0.2):
    test_set = ratings.copy() # Make a copy of the original set to be the test set. 
    test_set[test_set != 0] = 1 # Store the test set as a binary preference matrix
    training_set = ratings.copy() # Make a copy of the original data we can alter as our training set. 
    nonzero_inds = training_set.nonzero() # Find the indices in the ratings data where an interaction exists
    nonzero_pairs = list(zip(nonzero_inds[0], nonzero_inds[1])) # Zip these pairs together of user,item index into list
    random.seed(0) # Set the random seed to zero for reproducibility
    num_samples = int(np.ceil(pct_test*len(nonzero_pairs))) # Round the number of samples needed to the nearest integer
    samples = random.sample(nonzero_pairs, num_samples) # Sample a random number of user-item pairs without replacement
    user_inds = [index[0] for index in samples] # Get the user row indices
    item_inds = [index[1] for index in samples] # Get the item column indices
    training_set[user_inds, item_inds] = 0 # Assign all of the randomly chosen user-item pairs to zero
    training_set.eliminate_zeros() # Get rid of zeros in sparse array storage after update to save space
    return training_set, test_set, list(set(user_inds)) # Output the unique list of user rows that were altered  

In [8]:
product_train, product_test, product_users_altered = make_train(purchases_sparse, pct_test = 0.2)

In [9]:
item_lookup = pd.read_csv('cvs_data_file/item_lookup.csv', header = 0)
item_lookup.head()

,StockCode,Description
0,85123A,WHITE HANGING HEART T-LIGHT HOLDER
1,71053,WHITE METAL LANTERN
2,84406B,CREAM CUPID HEARTS COAT HANGER
3,84029G,KNITTED UNION FLAG HOT WATER BOTTLE
4,84029E,RED WOOLLY HOTTIE WHITE HEART.


In [15]:
def get_items_purchased(customer_id, mf_train, customers_list, products_list, item_lookup):
    cust_ind = np.where(customers_list == customer_id)[0][0] # Returns the index row of our customer id
    purchased_ind = mf_train[cust_ind,:].nonzero()[1] # Get column indices of purchased items
    prod_codes = products_list[purchased_ind] # Get the stock codes for our purchased items
    return item_lookup.loc[item_lookup.StockCode.isin(prod_codes)]

In [10]:
import pickle

with open('saved_model','rb') as f:
            saved_model=pickle.load(f)
            model=saved_model['model']

C:\Users\Office\Anaconda3\lib\site-packages\lightfm\_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn('LightFM was compiled without OpenMP support. '


In [132]:
# new
# item TO item recommendation
from sklearn.metrics.pairwise import cosine_similarity

def display_item_to_items_recommendations(model,item_id):

    products_arr = np.array(products) 
    
    print("products_arr",products_arr,str(item_id))
    item_id = np.where(products_arr == str(item_id))[0][0]
    print(item_id)
  
    return item_lookup['Description'][cosine_similarity(
            model.item_embeddings)[item_id].argsort()][-5:][::-1]


display_item_to_items_recommendations(model, 20719)

products_arr ['23166' '16008' '17021' ... '84613C' '84206B' '21414'] 20719
4


4           RED WOOLLY HOTTIE WHITE HEART.
289     3 PIECE SPACEBOY COOKIE CUTTER SET
948                  PORCELAIN ROSE LARGE 
2077              FRYING PAN BLUE POLKADOT
299       MAGIC DRAWING SLATE BAKE A CAKE 
Name: Description, dtype: object

In [130]:

def display_recommended_items(model, data, user_ids):
    customers_arr = np.array(customers)
    products_arr = np.array(products) 
    print("customers_arr",customers_arr)
    user_id = np.where(customers_arr == user_ids)[0][0]
    print(user_ids,user_id)
    
    n_users, n_items = data.shape
#     known_positives = get_items_purchased(user_ids, product_train, customers_arr, products_arr, item_lookup)
    known_positives = item_lookup['Description'][data.tocsr()[user_id].indices]
    known_positives_df = pd.DataFrame(data=known_positives)
    print(known_positives_df,'\n','######################################')            
    
    
    scores = model.predict(user_id, np.arange(n_items))

    print([np.argsort(-scores)])
    
    top_items = item_lookup['Description'][np.argsort(-scores)]
    print(top_items)            
            

display_recommended_items(model, product_train, 12347)


customers_arr [12346 12347 12348 ... 18282 18283 18287]
12347 1
                             Description
1                    WHITE METAL LANTERN
2         CREAM CUPID HEARTS COAT HANGER
3    KNITTED UNION FLAG HOT WATER BOTTLE
4         RED WOOLLY HOTTIE WHITE HEART.
5           SET 7 BABUSHKA NESTING BOXES
8              HAND WARMER RED POLKA DOT
9          ASSORTED COLOUR BIRD ORNAMENT
10            POPPY'S PLAYHOUSE BEDROOM 
11             POPPY'S PLAYHOUSE KITCHEN
13               IVORY KNITTED MUG COSY 
14    BOX OF 6 ASSORTED COLOUR TEASPOONS
15         BOX OF VINTAGE JIGSAW BLOCKS 
16        BOX OF VINTAGE ALPHABET BLOCKS
17              HOME BUILDING BLOCK WORD
18              LOVE BUILDING BLOCK WORD
21              JAM MAKING SET WITH JARS
22           RED COAT RACK PARIS FASHION
23        YELLOW COAT RACK PARIS FASHION
24          BLUE COAT RACK PARIS FASHION
25              BATH BUILDING BLOCK WORD
26             ALARM CLOCK BAKELIKE PINK
27             ALARM CLOCK BAKELIK